In [ ]:
import pandas as pd

document_set = pd.read_csv('document_set.csv')

In [ ]:
training_data = pd.read_csv('Training_Data.csv')

In [ ]:
data=training_data.merge(document_set,how='left',left_on='document_id',right_on='Document_Id')

In [ ]:
data=data[['Text','category']]

In [ ]:
import numpy as np
text=np.array(data['Text'])
label=np.array(data['category'])

In [ ]:
import nltk.data

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
import re
from nltk.corpus import stopwords

In [ ]:
def text_processing(doc):
    doc=[re.sub("[^a-zA-Z]"," ", s) for s in doc]
    doc=[s.lower().split() for s in doc]
    stops = set(stopwords.words("english"))
    doc=[[doc[i][j] for j in range(len(doc[i]))if not doc[i][j] in stops]for i in range(len(doc)) ]
    return doc

In [ ]:
text=text_processing(text)

In [ ]:
test_df = pd.read_csv(get_object_storage_file_with_credentials_24ed260965d844e2b4f006f0f948aa25('impoertant', 'Test_Data.csv'))


In [ ]:
test_set=test_df.merge(document_set,how='left',left_on='document_id',right_on='Document_Id')

In [ ]:
test_set=test_set[['document_id','Text']]

In [ ]:
test_text=text_processing(np.array(test_set['Text']))

In [ ]:

min_count=5
workers=4
window=10
sample = 1e-3 

In [ ]:
from gensim.models import word2vec
model = word2vec.Word2Vec(text, workers=workers, \
            size=num_features, min_count = min_count, \
            window = window, sample = sample)

In [ ]:
model.save('model')

In [ ]:
from gensim.models import Word2Vec

In [ ]:
features=300
def createFeature(doc, vectorModel, features):
    featureToVec = np.zeros((features,),dtype="float64")
    no_of_words=0
    index2word = set(model.wv.index2word)
    
    for word in doc:
        if word in index2word:
            no_of_words+=1
            featureToVec = np.add(featureToVec,vectorModel[word])
    featureToVec = np.divide(featureToVec,no_of_words)
    return featureToVec

In [ ]:
featureToVec = np.zeros((features,),dtype="float64")

In [ ]:
featureToVec.shape

In [ ]:
def getFeatureVector(doc, vectorModel, features):
    count=0
    featureToVecs=np.zeros((len(doc), features),dtype='float64')
    for d in doc:
        if count%2000==0:
            print('Text %d of %d done'%(count,len(doc)))
        featureToVecs[count]=createFeature(d,model,features)
        count +=1
    return featureToVecs

In [ ]:
train_vector_x=text[:-50]
test_vector_x=text[-50:]
label_x=label[:-50]
test_label=label[-50:]

In [ ]:
train_v_x=getFeatureVector(train_vector_x,model,features)

In [ ]:
test_v_x=getFeatureVector(test_vector_x,model,features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier( n_estimators = 100 )

In [ ]:
from sklearn.preprocessing import Imputer

trainDataVecs = Imputer().fit_transform(train_v_x)
testDataVecs=Imputer().fit_transform(test_v_x)

In [ ]:
clf=clf.fit(trainDataVecs,label_x)

In [ ]:
clf.score(testDataVecs,test_label)

In [ ]:
test_text_vec=getFeatureVector(test_text,model,features)

In [ ]:
test_text_vec=Imputer().fit_transform(test_text_vec)

In [ ]:
predict=clf.predict(test_text_vec)

In [ ]:
test_df.head()

In [ ]:
test_df_c=pd.DataFrame(test_df['document_id'])

In [ ]:
test_df_c.head()

In [ ]:
sub=pd.DataFrame(predict,columns=['category'])

In [ ]:
sub.head()

In [ ]:
result=test_df_c.join(sub,how='outer')

In [ ]:
result.head()

In [ ]:
result.to_csv('submit.csv',index=False)